# 使用 Google Gemini 和 LangChain 的卡牌效果生成器

本 notebook 演示如何使用 Google Gemini 模型通过 LangChain 来生成卡牌效果。

In [ ]:
# 导入必要的库
from typing import Dict, List, Any, Union, Optional
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.chains import LLMChain
from pydantic import BaseModel, Field, validator
import os

# 如果需要，安装必要的包
# !pip install langchain-google-genai google-generativeai

## 1. 设置环境变量
请确保设置了您的 Google API 密钥

In [ ]:
# 设置 Google API 密钥
# os.environ["GOOGLE_API_KEY"] = "your-api-key-here"

# 初始化 Gemini 模型
llm = GoogleGenerativeAI(
    model="gemini-pro",
    temperature=0.7,
    top_p=0.8,
    top_k=40,
    max_output_tokens=2048
)

## 2. 定义数据模型

In [ ]:
class CardEffect(BaseModel):
    effect_type: str = Field(..., description="效果类型，例如：伤害、治疗、buff等")
    target_type: str = Field(..., description="目标类型，例如：单体、全体、随机等")
    value: Union[int, str] = Field(..., description="效果值，可以是数值或特殊效果描述")
    condition: Optional[str] = Field(None, description="触发条件，如果有的话")
    duration: Optional[int] = Field(None, description="持续回合数，如果是持续效果")

class CommandOutput(BaseModel):
    card_id: str = Field(..., description="卡牌的唯一标识符")
    effect: CardEffect = Field(..., description="卡牌的效果详情")
    description: str = Field(..., description="卡牌效果的文字描述")

## 3. 设置提示模板和输出解析器

In [ ]:
# 创建输出解析器
parser = PydanticOutputParser(pydantic_object=CommandOutput)

# 创建提示模板
template = """你是一个卡牌游戏设计专家。请根据以下卡牌信息生成合适的卡牌效果：

卡牌名称: {card_name}
卡牌类型: {card_type}
费用: {cost}

要求：
1. 效果要平衡，不能过强或过弱
2. 效果要有趣且符合卡牌主题
3. 描述要清晰易懂

{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["card_name", "card_type", "cost"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

## 4. 创建 LangChain 链

In [ ]:
# 创建 LangChain 链
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
)

## 5. 生成卡牌效果示例

In [ ]:
# 示例卡牌信息
card_info = {
    "card_name": "冰霜巨龙",
    "card_type": "随从",
    "cost": 8
}

# 运行链并获取结果
response = chain.run(card_info)

# 解析响应
try:
    card_output = parser.parse(response)
    print("\n生成的卡牌效果：")
    print(f"卡牌ID: {card_output.card_id}")
    print("\n效果详情:")
    print(f"- 效果类型: {card_output.effect.effect_type}")
    print(f"- 目标类型: {card_output.effect.target_type}")
    print(f"- 效果值: {card_output.effect.value}")
    if card_output.effect.condition:
        print(f"- 触发条件: {card_output.effect.condition}")
    if card_output.effect.duration:
        print(f"- 持续回合: {card_output.effect.duration}")
    print(f"\n卡牌描述: {card_output.description}")
except Exception as e:
    print(f"解析错误: {e}")
    print(f"原始响应: {response}")

## 6. 批量生成多个卡牌效果

In [ ]:
# 批量生成多个卡牌效果
card_list = [
    {"card_name": "火焰术士", "card_type": "随从", "cost": 3},
    {"card_name": "治愈之光", "card_type": "法术", "cost": 2},
    {"card_name": "战争古树", "card_type": "随从", "cost": 5}
]

def generate_cards(card_list):
    results = []
    for card in card_list:
        try:
            response = chain.run(card)
            card_output = parser.parse(response)
            results.append(card_output)
        except Exception as e:
            print(f"处理卡牌 {card['card_name']} 时出错: {e}")
    return results

# 生成卡牌
generated_cards = generate_cards(card_list)

# 显示结果
for card in generated_cards:
    print(f"\n{'='*50}")
    print(f"卡牌: {card.card_id}")
    print(f"效果: {card.effect.effect_type} - {card.effect.value}")
    print(f"描述: {card.description}")